In [1]:
#source: https://medium.com/@erikhallstrm/tensorflow-rnn-api-2bb31821b185

In [2]:
#1. remove the W and b
#2. tanh operation of input_state is encapsulated

In [3]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [4]:

batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [batch_size, state_size])



W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

inputs_series = tf.split(1, truncated_backprop_length, batchX_placeholder)
labels_series = tf.unpack(batchY_placeholder, axis=1)

# Forward passes
cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
states_series, current_state = tf.nn.rnn(cell, inputs_series, init_state)

logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [ ]:
inputs_series[1].get_shape()

TensorShape([Dimension(5), Dimension(1)])

In [ ]:



def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)


with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        x,y = generateData()
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
New data, epoch 0
Step 0 Loss 0.705495


/home/wxk/anaconda2/lib/python2.7/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Step 100 Loss 0.0137861
Step 200 Loss 0.00584566
Step 300 Loss 0.0030437


/home/wxk/anaconda2/lib/python2.7/site-packages/matplotlib/axis.py:1025: UserWarning: Unable to find pixel distance along axis for interval padding of ticks; assuming no interval padding needed.
  warnings.warn("Unable to find pixel distance along axis "


Step 400 Loss 0.00218409
Step 500 Loss 0.00150646
Step 600 Loss 0.00207819
New data, epoch 1
Step 0 Loss 0.166399
Step 100 Loss 0.00104164
Step 200 Loss 0.000845079
Step 300 Loss 0.00118058
Step 400 Loss 0.000915893
Step 500 Loss 0.000766122
Step 600 Loss 0.000478764
New data, epoch 2
Step 0 Loss 0.1862
Step 100 Loss 0.000599721
Step 200 Loss 0.000507804
Step 300 Loss 0.000313763
Step 400 Loss 0.000433756
Step 500 Loss 0.000343777
Step 600 Loss 0.000291376
New data, epoch 3
Step 0 Loss 0.24828
Step 100 Loss 0.000617039
Step 200 Loss 0.000607483
Step 300 Loss 0.000538766
Step 400 Loss 0.000402344
Step 500 Loss 0.00037656
Step 600 Loss 0.000377558
New data, epoch 4
Step 0 Loss 0.118393
Step 100 Loss 0.000424071
Step 200 Loss 0.000325014
Step 300 Loss 0.000292978
Step 400 Loss 0.000211063
Step 500 Loss 0.000377548
Step 600 Loss 0.000258516
New data, epoch 5
Step 0 Loss 0.208503
Step 100 Loss 0.00033417
Step 200 Loss 0.000202602
Step 300 Loss 0.000145304
Step 400 Loss 0.000185119
Step 500 